# MIMIC-IV-on-FHIR Google Healthcare API Tutorial
This tutorial will walk through using MIMIC-IV-on-FHIR on GCP. The GCP Healthcare API provides the primary features all FHIR servers have.

The following features will be explored:
- Search
  - Search by gender
  - Search by condition
  - Search by medication
- Export
  - patient-everything 
  - subsets
- Aggregation?

Note:
- There seems to be some limitations to what the Google Healthcare API can do currently. _include is not working with the python api and extra search parameters are needed for some elements. (Ie medicationCodeableConcept)

In [1]:
from dotenv import load_dotenv
from pathlib import Path
import google.auth
from google.auth.transport import requests
from google.cloud import storage
import json
import time

In [2]:
load_dotenv(load_dotenv(Path(Path.cwd()).parents[0].parents[0] / '.env'))

True

In [3]:
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_TOPIC = os.getenv('GCP_TOPIC')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_BUCKET = os.getenv('GCP_BUCKET')
GCP_DATASET = os.getenv('GCP_DATASET')
GCP_FHIRSTORE = os.getenv('GCP_FHIRSTORE')
GCP_EXPORT_FOLDER = os.getenv('GCP_EXPORT_FOLDER')
GCP_TOPIC_PATIENT_EVERYTHING = os.getenv('GCP_TOPIC_PATIENT_EVERYTHING')

export_flag = True
graph_flag = True

credentials, project = google.auth.default()

In [4]:
session = requests.AuthorizedSession(credentials)
base_url = "https://healthcare.googleapis.com/v1"

project_url = f'{base_url}/projects/{GCP_PROJECT}/locations/{GCP_LOCATION}'
fhir_url = f'{project_url}/datasets/{GCP_DATASET}/fhirStores/{GCP_FHIRSTORE}/fhir'
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}



In [65]:
# export function 
# -- write out all the resources to the GCP export folder
def export_resources_to_storage(resources, resource_type, criteria, filter, pagenum=1, current_time=None):
    rlist = [json.dumps(rsrc['resource']) for rsrc in resources['entry']]
    output_bundle = '\n'.join(rlist)
    if current_time is None:
        current_time = time.strftime("%Y%m%d-%H%M%S")

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(GCP_BUCKET)
    filename = f"{GCP_EXPORT_FOLDER}/search/{current_time}-{resource_type}-{criteria}-{filter}/{pagenum}.ndjson"
    blob = bucket.blob(filename)
    blob.upload_from_string(output_bundle)

    link_info = [
        rsrc for rsrc in resources['link'] if rsrc['relation'] == 'next'
    ]
    if len(link_info) > 0:
        pagenum = pagenum + 1
        response = session.get(link_info[0]['url'], headers=headers)
        new_resources = response.json()
        export_resources_to_storage(new_resources, resource_type, criteria, filter, pagenum, current_time)
    return filename

# graph function
# -- simple graph with the number of resources with the metrics
# -- this seems less necessary, since most graphs will be one bar 
# maybe just have a summary print statement... just say X resources have Y criteria!
def print_search_results(resources, resource_type, criteria, filter):
    total_num = resources['total']
    print(f'SUMMARY RESULTS: {total_num} {resource_type} resources have {resource_type}.{criteria} equal to {filter}')

In [68]:
resources['link']

[{'relation': 'search',
  'url': 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/Patient/?gender=female'},
 {'relation': 'first',
  'url': 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/Patient/?gender=female'},
 {'relation': 'self',
  'url': 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/Patient/?gender=female'}]

## Search
- The _include functionality does not seem to be working, so you can search for a resource but cannot get any links to it (ie linked Patient to a Condition code)

### Search By Gender

In [67]:
resource_type = 'Patient'
gender = 'female'

resource_url = f'{fhir_url}/{resource_type}/_search?gender={gender}'
response = session.post(resource_url, headers=headers)
resources = response.json()
print_search_results(resources, resource_type, 'gender', gender)
export_resources_to_storage(resources, resource_type, 'gender', gender)

SUMMARY RESULTS: 6 Patient resources have Patient.gender equal to female


'mimic-iv-fhir-v2-export/search/20221115-145619-Patient-gender-female/1.ndjson'

### Search by Condition

In [21]:
resource_type = 'Condition'
code = '99591' #Sepsis

resource_url = f'{fhir_url}/{resource_type}/_search?code={code}&_include={resource_type}:subject'
response = session.post(resource_url, headers=headers)
response.raise_for_status()
resources = response.json()
print_search_results(resources, resource_type, 'code', code)

SUMMARY RESULTS: 1 Condition resources have Condition.code equal to 99591


### Search by Procedure

In [23]:
resource_type = 'Procedure'
code = '227194' #Extubation

resource_url = f'{fhir_url}/{resource_type}?code={code}&_include={resource_type}:subject'
response = session.get(resource_url, headers=headers)
response.raise_for_status()
resources = response.json()
print_search_results(resources, resource_type, 'code', code)

SUMMARY RESULTS: 4 Procedure resources have Procedure.code equal to 227194


### Search by Medication

In [110]:
resource_type = 'MedicationAdministration'
code = 'NACLFLUSH' #Extubation

#resource_url = f'{fhir_url}/{resource_type}/_search?medicationCodeableConcept.coding.code={code}'
resource_url = f'{fhir_url}/{resource_type}/_search?status=completed'
response = session.post(resource_url, headers=headers)
response.raise_for_status()
resources = response.json()
print_search_results(resources, resource_type, 'code', code)
#export_resources_to_storage(resources, resource_type, 'code', code)

SUMMARY RESULTS: 692 MedicationAdministration resources have MedicationAdministration.code equal to NACLFLUSH


In [70]:
resources['link']

[{'relation': 'search',
  'url': 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/MedicationAdministration/?'},
 {'relation': 'next',
  'url': 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/MedicationAdministration/?&_page_token=Cjj3Y0b37v%2F%2F%2F%2F%2BABerZpKgOUwD%2FAf%2F%2BZWJiOGZjMDkzODkwYWJjMjg1OTRmNTM0ZWUwMjhmODMAARBkIfp7n2jesxm7OQAAAAARCLmcSBZQAFoLCcZS1glp7AyvEANg6ILyiQE%3D'},
 {'relation': 'first',
  'url': 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/MedicationAdministration/?'},
 {'relation': 'self',
  'url': 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/MedicationAdministration/?'}

In [98]:
link_list=[]
link_list.append(resources['link'][1]['url'])

In [105]:
response = session.get(resources['link'][1]['url'], headers=headers)
resources = response.json()
resources['link']
link_list.append(resources['link'][1]['url'])

In [106]:
link_list

['https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/MedicationAdministration/?&_page_token=Cjj3Y0b37v%2F%2F%2F%2F%2BABerZpKgOUwD%2FAf%2F%2BZWJiOGZjMDkzODkwYWJjMjg1OTRmNTM0ZWUwMjhmODMAARBkIfp7n2jesxm7OQAAAAARCLmcSBZQAFoLCcZS1glp7AyvEANg3J7BvQE%3D',
 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/MedicationAdministration/?&_page_token=Cjj3Y0b37v%2F%2F%2F%2F%2BABerZpKgOUwD%2FAf%2F%2BN2U4YTExZDNkNWY2YzRkMzcxZmM2ZTgzYzZmOGNmMmIAARDIASH6e59o3rMZuzkAAAAAEQi5nEh6UABaCwnGUtYJaewMrxADYNyewb0B',
 'https://healthcare.googleapis.com/v1/projects/kind-lab/locations/us-central1/datasets/mimic-iv-fhir-dataset/fhirStores/mimic-iv-fhir-v2-demo/fhir/MedicationAdministration/?&_page_token=Cjj3Y0b37v%2F%2F%2F%2F%2BABerZpKgOUwD%2FAf%2F%2BMTJiNWUyOGRmZjljYjUzYmMwZjc3Y2JjMjJjZTBhODUAARCsAiH6e59o3rMZuzkAAAAAEQi5nEjeAVA

### Get Capability Statement

In [ ]:
resource_url = f'{fhir_url}/metadata'
response = session.get(resource_url, headers=headers)
response.raise_for_status()
resources = response.json()
resources

with open('capability.json', 'w+') as f:
    json.dump(resources, f)

## Export

In [ ]:
# Support functions
def get_resource_ids(fhir_url, resource_type):
    resource_url = f'{fhir_url}/{resource_type}/_search?_elements=id'
    response = session.post(resource_url, headers=headers)
    response.raise_for_status()
    resources = response.json()
    patient_ids = [ entry['resource']['id'] for entry in resources['entry']]
    return patient_ids

### Patient-everything

In [ ]:
GCP_PATIENT_EVERYTHING_FOLDER = f'patient-everything/bundles-{time.strftime("%Y%m%d-%H%M%S")}'

In [ ]:
def send_patient_everything(export_url, headers, patient_id, page_num=1):
    response = session.get(export_url, headers=headers)
    response.raise_for_status()
    resp_fhir = response.json()

    if 'error' in resp_fhir:
        print('ERROR IN RESPONSE')
    elif resp_fhir['resourceType'] == 'OperationOutcome':
        print(resp_fhir['issue'][0])
    elif  ((resp_fhir['resourceType'] == 'Bundle') and ('link' in resp_fhir)):
        filename = export_bundle_to_storage(resp_fhir, patient_id, page_num)
        print(f'Stored file: {filename}')
        link_info = [
            resp for resp in resp_fhir['link'] if resp['relation'] == 'next'
        ]
        if len(link_info) > 0:
            send_patient_everything(
                link_info[0]['url'], headers, patient_id, page_num + 1
            )
    else:
        filename = export_bundle_to_storage(resp_fhir, patient_id, page_num)
        print(f'Stored file: {filename}')


    return resources

def export_bundle_to_storage(resp_fhir, patient_id, page_num):
    bundle = resp_fhir

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(GCP_BUCKET)
    filename = f"{GCP_PATIENT_EVERYTHING_FOLDER}/patient-{patient_id}-page{page_num}"
    blob = bucket.blob(filename)
    blob.upload_from_string(json.dumps(bundle))
    return filename

In [ ]:
# Get some patient_ids

resource_type = 'Patient'
output_resource_types = 'Patient,Encounter,Condition,Procedure' # resource types to output
num_patients = 1
count = 100 # how many resources per bundle page

patient_list = get_resource_ids(fhir_url, resource_type)
patient_list

if num_patients > len(patient_list):
    num_patients = len(patient_list)
for idx in range(0,num_patients):
    patient_id = patient_list[idx]
    export_url = f'{fhir_url}/Patient/{patient_id}/$everything?_count={count}&_type={output_resource_types}'
    print(export_url)
    resources = send_patient_everything(export_url, headers, patient_id)